**MỤC TIÊU TỔNG QUÁT**

**1. Xử lý dữ liệu đầu vào:**

Xử lý giá trị thiếu cho các cột số và cột phân loại.
Mã hóa các cột phân loại thành dạng số để mô hình có thể sử dụng.
Chuẩn hóa dữ liệu để cải thiện hiệu suất mô hình.

**2. Huấn luyện mô hình:**

Sử dụng thuật toán Random Forest Classifier để xây dựng mô hình.
Thực hiện đánh giá hiệu suất bằng Cross-Validation.
Lưu lại các báo cáo chi tiết về hiệu suất mô hình.

**3. Tối ưu hóa mô hình:**

Dùng GridSearchCV để tìm kiếm các tham số tối ưu.
Đảm bảo rằng mô hình đạt được hiệu suất cao nhất trên tập dữ liệu huấn luyện.

**4. Lưu trữ và sử dụng lại:**

Lưu mô hình, scaler (chuẩn hóa dữ liệu), label encoder (mã hóa nhãn) để tái sử dụng trong tương lai.
Lưu lại các kết quả dự đoán từ tập dữ liệu kiểm tra.

**5. Ghi log và báo cáo:**

Ghi lại quá trình thực thi vào file log, giúp dễ dàng theo dõi hoặc debug.
Tạo báo cáo chi tiết về mô hình (bằng Markdown), bao gồm hiệu suất và tham số.

Đoạn code sau thực hiện:
*   Kết nối Google Drive với Google Colab để truy cập dữ liệu lưu trữ trong Drive.
*   Tải file lên từ máy tính cá nhân nếu dữ liệu không có sẵn trên Google Drive.
*   Đọc file CSV từ Google Drive bằng thư viện pandas để chuẩn bị dữ liệu cho các bước xử lý hoặc phân tích tiếp theo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
# uploaded = files.upload()
# Sau khi tải file lên, đọc file bằng pandas
import pandas as pd
df = pd.read_csv('drive/MyDrive/ML/RandomForest/train.csv')

In [ ]:
from google.colab import files

# Sau khi tải file lên, đọc file bằng pandas
import pandas as pd
df = pd.read_csv('drive/MyDrive/ML/RandomForest/test.csv')

Import các thư viện

*   Sử dụng pandas và numpy để xử lý dữ liệu
*   sklearn để xây dựng và đánh giá mô hình machine learning
*   joblib để lưu/tải mô hình
*   logging để ghi log quá trình training
*   pathlib và os để xử lý đường dẫn file
*   warnings để kiểm soát cảnh báo







In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import logging
from pathlib import Path
import datetime
import os
import warnings

Khởi tạo lớp MLPipeline

Lớp MLPipeline là trung tâm của pipeline này, đảm nhiệm các công việc từ xử lý dữ liệu, huấn luyện, tối ưu hóa mô hình đến lưu trữ kết quả.

1. Hàm setup_logging():
Đảm bảo mọi thông tin quan trọng trong quá trình thực thi được lưu trữ và theo dõi.

2. Hàm load_data():
Đảm bảo rằng dữ liệu đầu vào từ các file CSV được kiểm tra, đọc đúng định dạng và trả về kết quả hợp lệ.

3. Hàm preprocess_data():
Xử lý dữ liệu một cách hệ thống, bao gồm điền giá trị thiếu, chuẩn hóa và mã hóa.

4. Hàm train_random_forest():
Huấn luyện mô hình cơ bản và đánh giá hiệu suất bằng Cross-Validation.

5. Hàm optimize_random_forest():
Tìm tham số tối ưu cho mô hình Random Forest thông qua GridSearchCV.

6. Hàm save_artifacts():
Quản lý và lưu trữ các artifacts như mô hình, scaler, label encoder và dự đoán.

7. Hàm run_pipeline():
Chạy toàn bộ pipeline từ đầu đến cuối, kết hợp các bước trên một cách tự động.

In [ ]:
class MLPipeline:
    def __init__(self, random_state=42, n_jobs=None):
        #random_state=42: Đặt seed ngẫu nhiên để đảm bảo kết quả có thể tái hiện.
        #n_jobs=None: Số luồng (CPU cores) để chạy. Nếu không được cung cấp, nó mặc định là một nửa số nhân CPU.
        #self.setup_logging(): Gọi hàm thiết lập logging để lưu trữ nhật ký (logs).
        self.random_state = random_state
        self.n_jobs = n_jobs if n_jobs is not None else max(os.cpu_count() // 2, 1)
        self.setup_logging()

    #Thiết lập logging với cấu hình ghi thông tin vào tệp và hiển thị trên console, sử dụng timestamp để lưu tên file
    def setup_logging(self):
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(f'training_{timestamp}.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    # Tạo report đánh giá (bao gồm các bước thực hiện)
    def generate_evaluation_report(self, model, cv_scores, best_params=None):
        """
        Generate a comprehensive evaluation report for the model.
        """
        report = """
# Random Forest Model Evaluation Report

## 1. Model Performance Metrics
- Mean Cross-Validation Score: {:.4f} (+/- {:.4f})
- Cross-Validation Scores: {}

## 2. Model Configuration
### Base Model Parameters:
- Number of Trees (n_estimators): {}
- Maximum Depth: {}
- Number of CPU Cores Used: {}

""".format(
            cv_scores.mean(),
            cv_scores.std() * 2,
            [f"{score:.4f}" for score in cv_scores],
            model.n_estimators,
            model.max_depth,
            self.n_jobs
        )

        if best_params:
            report += """
### Optimized Parameters:
{}
""".format('\n'.join([f"- {k}: {v}" for k, v in best_params.items()]))

        # Lưu report
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        report_path = f'model_evaluation_{timestamp}.md'
        with open(report_path, 'w') as f:
            f.write(report)

        self.logger.info(f"Evaluation report saved to {report_path}")
        return report

    # load data thành tập train và test
    def load_data(self, train_file: str, test_file: str, sample_fraction: float = 0.1) -> tuple:
        try:
            df_train = pd.read_csv(train_file)
            df_test = pd.read_csv(test_file)

            if 'Label' not in df_train.columns:
                raise ValueError("Training data must contain 'Label' column")

            train_features = set(df_train.columns) - {'Label'}
            test_features = set(df_test.columns)
            if train_features != test_features:
                raise ValueError("Training and test data must have matching features")

            self.logger.info(f"Loaded training data shape: {df_train.shape}")
            self.logger.info(f"Loaded test data shape: {df_test.shape}")

            return df_train, df_test

        except FileNotFoundError as e:
            self.logger.error(f"File not found: {e}")
            raise
        except Exception as e:
            self.logger.error(f"Error loading data: {e}")
            raise

    # tiền xử lý data
    def preprocess_data(self, df: pd.DataFrame, scaler=None, label_encoder=None,
                        is_training=True) -> tuple:
        try:
            df = df.copy()

            # Xác định loại cột
            numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
            categorical_cols = df.select_dtypes(include=['object']).columns

            # Xử lý missing values cho từng loại dữ liệu
            # Numeric: điền bằng median
            for col in numeric_cols:
                if is_training:
                    median_val = df[col].median()
                    df[col] = df[col].fillna(median_val)
                    if hasattr(self, f'median_{col}'):
                        setattr(self, f'median_{col}', median_val)
                else:
                    if hasattr(self, f'median_{col}'):
                        df[col] = df[col].fillna(getattr(self, f'median_{col}'))
                    else:
                        df[col] = df[col].fillna(0)

            # Categorical: điền bằng mode
            for col in categorical_cols:
                if is_training:
                    mode_val = df[col].mode()[0]
                    df[col] = df[col].fillna(mode_val)
                    if hasattr(self, f'mode_{col}'):
                        setattr(self, f'mode_{col}', mode_val)
                else:
                    if hasattr(self, f'mode_{col}'):
                        df[col] = df[col].fillna(getattr(self, f'mode_{col}'))
                    else:
                        df[col] = df[col].fillna('Unknown')

            # Chuyển đổi dữ liệu phân loại (chuỗi) thành số
            for col in categorical_cols:
                if is_training:
                    le = LabelEncoder()
                    df[col] = le.fit_transform(df[col].astype(str))
                    if hasattr(self, f'le_{col}'):
                        setattr(self, f'le_{col}', le)
                else:
                    if hasattr(self, f'le_{col}'):
                        le = getattr(self, f'le_{col}')
                        df[col] = df[col].astype(str)
                        # Handle unseen categories
                        df[col] = df[col].map(lambda x: 'Unknown' if x not in le.classes_ else x)
                        df[col] = le.transform(df[col])
                    else:
                        df[col] = df[col].astype('category').cat.codes

            # Chuyển đổi dữ liệu sang float32 để tối ưu bộ nhớ
            for col in numeric_cols:
                df[col] = df[col].astype('float32')

            if is_training:
                X = df.drop('Label', axis=1)
                y = df['Label']

                label_encoder = LabelEncoder()
                scaler = StandardScaler()

                # Sử dụng StandardScaler để chuẩn hóa dữ liệu
                y = label_encoder.fit_transform(y)
                X = scaler.fit_transform(X)

                self.logger.info(f"Preprocessed training data shape: {X.shape}")
                return X, y, scaler, label_encoder
            else:
                X = scaler.transform(df)
                self.logger.info(f"Preprocessed test data shape: {X.shape}")
                return X, None

        except Exception as e:
            self.logger.error(f"Error in preprocessing: {e}")
            raise

    def train_random_forest(self, X: np.ndarray, y: np.ndarray) -> tuple:
        try:
            # chia dữ liệu
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=0.2, random_state=self.random_state
            )

            #n_estimators=100: Số cây trong rừng
            #max_depth=30: Độ sâu tối đa của cây
            #warm_start=True: Tiếp tục huấn luyện từ trạng thái trước đó
            #Sử dụng 80% dữ liệu huấn luyện được sử dụng để huấn luyện từng cây, giúp giảm tương quan giữa các cây.
            model = RandomForestClassifier(
                n_estimators=100,
                random_state=self.random_state,
                n_jobs=self.n_jobs,
                verbose=1,
                max_depth=30,
                max_samples=0.8,
                warm_start=True
            )

            # Cross-validation
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                cv_scores = cross_val_score(
                    model, X, y,
                    cv=5,
                    n_jobs=self.n_jobs,
                    verbose=1
                )

            self.logger.info(f"CV Scores: {cv_scores}")
            self.logger.info(f"Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

            # Huấn luyện mô hình
            model.fit(X_train, y_train)

            # Dự đoán trên tập đánh giá bằng accuracy
            val_predictions = model.predict(X_val)
            val_accuracy = accuracy_score(y_val, val_predictions)

            self.logger.info(f"Validation Accuracy: {val_accuracy:.4f}")
            self.logger.info("\nClassification Report:")
            self.logger.info(classification_report(y_val, val_predictions))

            # tạo report
            self.generate_evaluation_report(model, cv_scores)

            return model, cv_scores.mean()

        except Exception as e:
            self.logger.error(f"Error in training: {e}")
            raise

    def optimize_random_forest(self, X: np.ndarray, y: np.ndarray) -> RandomForestClassifier:
        try:
            # Các tham số có thể tối ưu:
            #n_estimators: Số lượng cây cần thử: 200
            #max_depth: Giới hạn độ sâu của cây: 20 hoặc không giới hạn
            #min_samples_split: Số lượng mẫu tối thiểu cần có để tránh overfitting: 5
            #min_samples_leaf: Số lượng mẫu tối thiểu cần có trong một lá: 2
            #max_features: Số lượng đặc trưng tối đa xem xét khi tách tại mỗi node
            #'sqrt' được sử dụng để tăng tốc và giảm tương quan giữa các cây.
            param_grid = {
                'n_estimators': [200],
                'max_depth': [20, None],
                'min_samples_split': [5],
                'min_samples_leaf': [2],
                'max_features': ['sqrt']
            }

            base_model = RandomForestClassifier(
                random_state=self.random_state,
                n_jobs=self.n_jobs,
                verbose=1
            )

            # Thử nghiệm từng tham số trong param_grid bằng cách sử dụng cross-validation với 3 fold.
            grid_search = GridSearchCV(
                base_model,
                param_grid,
                cv=3,
                n_jobs=self.n_jobs,
                scoring='accuracy',
                verbose=2
            )

            # Huấn luyện mô hình với tất cả tham số trong param_grid và chọn ra tham số tốt nhất
            grid_search.fit(X, y)

            self.logger.info(f"Best parameters: {grid_search.best_params_}")
            self.logger.info(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

            # Tạo report
            self.generate_evaluation_report(
                grid_search.best_estimator_,
                cross_val_score(grid_search.best_estimator_, X, y, cv=5),
                grid_search.best_params_
            )

            return grid_search.best_estimator_

        except Exception as e:
            self.logger.error(f"Error in optimization: {e}")
            raise

    # Lưu các thành phần quan trọng của pipeline (mô hình, scaler, label encoder, và dự đoán) vào các tệp
    def save_artifacts(self, model, scaler, label_encoder, predictions,
                       predictions_labels, output_dir='outputs'):
        try:
            output_dir = Path(output_dir)
            output_dir.mkdir(exist_ok=True)

            timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

            joblib.dump(model,
                        output_dir / f'model_{timestamp}.joblib',
                        compress=3)
            joblib.dump(scaler,
                        output_dir / f'scaler_{timestamp}.joblib',
                        compress=3)
            joblib.dump(label_encoder,
                        output_dir / f'label_encoder_{timestamp}.joblib',
                        compress=3)

            results_df = pd.DataFrame({
                'Predicted_Label_Encoded': predictions,
                'Predicted_Label': predictions_labels
            })
            results_df.to_csv(output_dir / f'predictions_{timestamp}.csv',
                              index=False)

            self.logger.info(f"All artifacts saved to {output_dir}")

        except Exception as e:
            self.logger.error(f"Error saving artifacts: {e}")
            raise

    def run_pipeline(self, train_file: str, test_file: str, output_dir='outputs'):
        try:
            self.logger.info("Starting ML pipeline")

            # load dữ liệu train và test
            df_train, df_test = self.load_data(train_file, test_file)

            # tiền xử lý dữ liệu train
            X_train, y_train, scaler, label_encoder = self.preprocess_data(
                df_train, is_training=True
            )

            # train model
            model, base_score = self.train_random_forest(X_train, y_train)

            # optimize model
            best_model = self.optimize_random_forest(X_train, y_train)

            # tiền xử lý dữ liệu test
            X_test, _ = self.preprocess_data(
                df_test, scaler, label_encoder, is_training=False
            )

            # thực hiện dự đoán trên tập test
            predictions = best_model.predict(X_test)
            predictions_labels = label_encoder.inverse_transform(predictions)

            # lưu tham số
            self.save_artifacts(
                best_model, scaler, label_encoder,
                predictions, predictions_labels, output_dir
            )

            return best_model, scaler, label_encoder

        except Exception as e:
            self.logger.error(f"Pipeline failed: {e}")
            raise


Khởi động và chạy toàn bộ pipeline Machine Learning (MLPipeline) từ dữ liệu đầu vào cho đến kết quả cuối cùng.

In [ ]:
if __name__ == "__main__":
    n_jobs = max(os.cpu_count() // 2, 1)

    # Khởi tạo và chạy pipeline
    pipeline = MLPipeline(n_jobs=n_jobs)
    try:
        best_model, scaler, label_encoder = pipeline.run_pipeline(
            train_file='drive/MyDrive/ML/RandomForest/train.csv',
            test_file='drive/MyDrive/ML/RandomForest/test.csv'
        )
    except Exception as e:
        print(f"Pipeline failed: {e}")